In [ ]:
# Testing plotting tools (geographic information)
# https://matplotlib.org/basemap/users/index.html
# https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f
# https://www.datacamp.com/community/tutorials/geospatial-data-python
# https://towardsdatascience.com/mapping-with-matplotlib-pandas-geopandas-and-basemap-in-python-d11b57ab5dac

In [ ]:
# Plotting tools
# Matplotlib (with Basemap)
# Plotly
# Altair
# Folium (aka OSM interativo)
# Geopandas

In [1]:
import boto3
import os
SECRET_KEY="your_key"
ACCESS_KEY="your_key"

In [4]:
bucketName='mobility-traces-bus-sp'
remoteDirectoryName='partioned-day-bus'

import boto3
import os

s3_client = boto3.client('s3',aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY)

# Download all the files from a bucket and determined prefix (folder)
# https://stackoverflow.com/questions/31918960/boto3-to-download-all-files-from-a-s3-bucket/31929277
def download_dir(prefix=remoteDirectoryName, local="./saida", bucket=bucketName, client=s3_client):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)
download_dir()

In [47]:
# https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/
import pandas as pd
import glob

# bus 11332, day 1/10/2015
path = r'./sample-bus-traces/working-day/11332' # use your path, id_avl = 11332
all_files = glob.glob(path + "/*.csv")

li = []
column_names = "dt_server,dt_avl,id_line,longitude,latitude,id_avl,event,id_point".split(",")

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=None)
    df.columns=column_names
    li.append(df)

df_11332 = pd.concat(li)

df_11332_sorted = df_11332.sort_values(by=['dt_avl'])
df_11332_sorted.to_csv("sorted.csv")

In [48]:
df_11332_sorted.head(10)

,dt_server,dt_avl,id_line,longitude,latitude,id_avl,event,id_point
0,2015-10-01 04:00:25.977,2015-10-01 04:00:24.000,1175,-23.684007,-46.714540,11332,64,9975
1,2015-10-01 04:00:57.937,2015-10-01 04:00:56.000,1175,-23.688160,-46.714687,11332,64,9976
2,2015-10-01 04:01:39.837,2015-10-01 04:01:38.000,1175,-23.693670,-46.714903,11332,64,9977
3,2015-10-01 04:02:35.963,2015-10-01 04:02:35.000,1175,-23.700562,-46.715140,11332,64,0
4,2015-10-01 04:02:55.923,2015-10-01 04:02:38.000,1175,-23.701052,-46.715133,11332,64,9978
5,2015-10-01 04:03:00.810,2015-10-01 04:02:57.000,1175,-23.703375,-46.714902,11332,64,0
6,2015-10-01 04:05:40.990,2015-10-01 04:03:43.000,1175,-23.708912,-46.712205,11332,64,0
9,2015-10-01 04:05:46.367,2015-10-01 04:03:59.000,1175,-23.710760,-46.711300,11332,64,5424
7,2015-10-01 04:05:41.240,2015-10-01 04:04:42.000,1175,-23.714355,-46.707983,11332,64,0
10,2015-10-01 04:05:46.640,2015-10-01 04:04:42.000,1175,-23.714355,-46.707983,11332,64,0


In [49]:
# Testing Folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

In [46]:
# bus 11332
locations_11332 = df_11332[['longitude','latitude']]
locationlist = locations_11332.values.tolist()
map = folium.Map(location=[-23.533773, -46.625290], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point]).add_to(map)

folium.PolyLine(locationlist, color=None, weight=2.5, opacity=1).add_to(map)

In [37]:
map

In [51]:
df_11332_sorted.groupby('event').nunique()

,dt_server,dt_avl,id_line,longitude,latitude,id_avl,event,id_point
event,,,,,,,,
0,822,803,2,417,419,1,1,48
64,1202,1185,2,867,898,1,1,75


In [62]:
# https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/
import pandas as pd
import glob

# bus 44261, day 4/10/2015
path = r'./sample-bus-traces/weekend/44261' # use your path, id_avl = 44261
all_files = glob.glob(path + "/*.csv")

li = []
column_names = "dt_server,dt_avl,id_line,longitude,latitude,id_avl,event,id_point".split(",")

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=None)
    df.columns=column_names
    li.append(df)

df_44261 = pd.concat(li)

df_44261_sorted = df_44261.sort_values(by=['dt_avl'])
df_44261_sorted.to_csv("sorted.csv")

In [63]:
# bus 44261
locations_44261 = df_44261[['longitude','latitude']]
locationlist = locations_44261.values.tolist()
map = folium.Map(location=[-23.533773, -46.625290], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point]).add_to(map)

folium.PolyLine(locationlist, color=None, weight=2.5, opacity=1).add_to(map)

In [65]:
map

In [66]:
df_44261_sorted.groupby('event').nunique()

,dt_server,dt_avl,id_line,longitude,latitude,id_avl,event,id_point
event,,,,,,,,
0,1422,1422,1,91,57,1,1,1
64,476,476,1,187,158,1,1,1


In [67]:
# https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/
import pandas as pd
import glob

# bus 51256, day 1/10/2015
path = r'./sample-bus-traces/working-day/51256' # use your path, id_avl = 44261
all_files = glob.glob(path + "/*.csv")

li = []
column_names = "dt_server,dt_avl,id_line,longitude,latitude,id_avl,event,id_point".split(",")

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=None)
    df.columns=column_names
    li.append(df)

df_51256 = pd.concat(li)

df_51256_sorted = df_51256.sort_values(by=['dt_avl'])
df_51256_sorted.to_csv("sorted.csv")

In [68]:
# bus 51256
locations_51256 = df_51256[['longitude','latitude']]
locationlist = locations_51256.values.tolist()
map = folium.Map(location=[-23.533773, -46.625290], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point]).add_to(map)

folium.PolyLine(locationlist, color=None, weight=2.5, opacity=1).add_to(map)

In [69]:
map

In [70]:
df_51256_sorted.groupby('event').nunique()

,dt_server,dt_avl,id_line,longitude,latitude,id_avl,event,id_point
event,,,,,,,,
0,627,604,2,347,308,1,1,3
64,1561,1409,3,1053,1012,1,1,4


In [71]:
#Tipos de erros a serem tratados/identificar para separar trajetorias
#- onibus pode estar parados ha horas no mesmo lugar
#- identificar quando termina quando comeca trajetoria
#- selecionar só area de SP 
#- eliminar dados madrugada
#- quando tempo entre pontos consecutivos for maior que x segundos
#- refazer metricas pra ver se tem relacao 64 e 0 (evento)

In [ ]:
# Mapping Matching
# https://github.com/gboeing/osmnx/issues/315
# https://github.com/ustroetz/python-osrm
# https://github.com/valhalla/valhalla/blob/master/docs/meili/algorithms.md
# https://github.com/bmwcarit/barefoot
# https://labs.strava.com/slide/
# https://medium.com/driving-to-the-future/map-matching-and-the-processing-of-raw-gps-data-on-an-industrial-scale-599a9475d332
# https://github.com/Project-OSRM/osrm-backend/
# https://medium.com/driving-to-the-future/map-matching-and-the-processing-of-raw-gps-data-on-an-industrial-scale-599a9475d332
# https://pypi.org/project/mapbox/
# https://github.com/mapbox/mapbox-sdk-py/blob/master/docs/mapmatching.md